In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../zkstats/core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
def mode_within(data_array, percent):
    max_sum_freq = 0
    mode = data_array[0]

    for check_val in set(data_array):
        sum_freq = sum(1 for ele in data_array if abs(ele - check_val) <= abs(percent * check_val / 100))

        if sum_freq > max_sum_freq:
            mode = check_val
            max_sum_freq = sum_freq

    return mode

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(mode_within(dummy_data,1))




In [6]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(4.6000, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_44922/1447245977.py:10: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_44922/1447245977.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [7]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(mode_within(data,1))

print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])
        return (torch.sum(result) == X.size()[1], self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_44922/436594305.py:14: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


Theory_output:  tensor(77.)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":9,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[9],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_44922/436594305.py:14: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor([torch.sum((torch.abs(X-ele[0])<=torch.abs(0.01*ele[0])).double())<=count_equal for ele in X[0]])


In [8]:
settings_read = json.load(open(settings_path, 'r'))
print(settings_read)
print(type(settings_read['run_args']['logrows']))

{'run_args': {'tolerance': {'val': 0.0, 'scale': 1.0}, 'input_scale': 9, 'param_scale': 9, 'scale_rebase_multiplier': 10, 'lookup_range': [0, 0], 'logrows': 14, 'num_inner_cols': 2, 'variables': [['batch_size', 1]], 'input_visibility': {'Hashed': {'hash_is_public': True, 'outlets': []}}, 'output_visibility': 'Public', 'param_visibility': 'Private'}, 'num_rows': 14432, 'total_assignments': 300, 'total_const_size': 0, 'model_instance_shapes': [[1], [1]], 'model_output_scales': [0, 9], 'model_input_scales': [9], 'module_sizes': {'kzg': [], 'poseidon': [14432, [1]], 'elgamal': [0, [0]]}, 'required_lookups': [], 'check_mode': 'UNSAFE', 'version': '7.0.0', 'num_blinding_factors': None}
<class 'int'>


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2
spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0


Time setup: 1.3558268547058105 seconds
Theory output:  tensor(77.)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 77.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[8148670344449872040, 3850172417927569887, 9386325907200828332, 1904490596485556704], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7029749938363617618, 11538692499682334571, 8918902057354509017, 3053834031406807957]]], 'proof': '178f3cbb464930eeb074e60aae9f1629dca90b16e81612ca19781fee360054ed0fa5a8b1693ce5df3938e36490f12ec74135c8a04bdae3896e8f327f7500e4cb1fdf6d74557a78e312d902a08f00ade8fb0c6380d1ec7ed932507e3fa63baf74126420f20dfad37b8b9339ed9677994149920f445ec2387d5642c386b515d1180d6127eaa4e659034f25f62d114263691ff09907b18d128aa79c36cf8335ad3e2224e03970e5a0ddd5fad56713c893f213a7f6da80fa92f9e4c274c8c259beeb15438ce6eb717775503cd3a05e611284be196985a7d8925d1aa93ef88dbdf162095a0f8e0745cfee97b3a1ae106ce8290eff0bb7292a0362a05274ea224c16300b9c4ea0c4ac7b434b6a1e335cce0f5f89abde37b9ad96a560f8aa94977f23d5017bca2160a262b93e0fc73a59a231d77ec21163750d9b56fe05de362fed75b328ef5abd13a756ee9165fb2ac25fab2eaa9434e977d3471cd8b80f4bec3027ba

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[8148670344449872040, 3850172417927569887, 9386325907200828332, 1904490596485556704], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7029749938363617618, 11538692499682334571, 8918902057354509017, 3053834031406807957]]]
proof boolean:  1.0
proof result 1 : 77.0
verified
